# Algorytm RSA

Obecnie najpopularniejszy algorytm kryptografi asymetrycznej. Opublikowany w 1977 roku w *Scientific American's Mathematical Games* przez Rona Rivesta, Adiego Shamira i Leonarda Adlemana. Opiera się na problemie faktoryzacji liczb.

W uproszczonej wersji algorytm RSA wygląda następująco:

### Generowanie kluczy

- Wybieramy dwie losowe liczby pierwsze $p$ i $q$
- Obliczamy $n=pq$
- Obliczamy funkcję Eulera $\phi(n)=(p-1)(q-1)$
- Wybieramy $e\in\{2,...,\phi(n)-1\}$ względnie pierwszą z $\phi(n)$
- Obliczamy $d=e^{-1}\mod \phi(n)$
- Klucz publiczny to para $(n,e)$ a klucz prywatny to para $(n,d)$.


### Szyfrowanie
Mamy dany klucz publiczny $(n,e)$. Wiadomość szyfrowana jest liczbą $m\in [0,n)$.
- Obliczamy szyfrogram $$c=m^e\mod n.$$

Teoretycznie może się zdarzyć, że szyfrogram $c$ jest taki sam, jak szyfrowana wiadomość $m$.

### Deszyfrowanie
- Obliczamy odszyfrowaną wiadomość $$m=c^d=m^{ed}\mod n$$

In [1]:
import random
from math import gcd

### Ćwiczenie 1.

Zaimplementuj uproszczony algorytm RSA.

In [2]:
# Simple primality test
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Generate random prime
def generate_prime(bit_length):
    while True:
        num = random.randint(2**(bit_length-1), 2**bit_length - 1)
        if is_prime(num):
            return num

# Extended Euclidean algorithm
def mod_inverse(e, phi):
    def egcd(a, b):
        if a == 0:
            return b, 0, 1
        else:
            g, y, x = egcd(b % a, a)
            return g, x - (b // a) * y, y
    g, x, _ = egcd(e, phi)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % phi

def rsa_keygen(bit_length=16):
    p = generate_prime(bit_length // 2)
    q = generate_prime(bit_length // 2)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    e = 65537
    if gcd(e, phi_n) != 1:
        e = 3  # fallback if 65537 not coprime with phi_n
    d = mod_inverse(e, phi_n)
    return (n, e), (n, d)

def rsa_encrypt(message, public_key):
    n, e = public_key
    return pow(message, e, n)

def rsa_decrypt(ciphertext, private_key):
    n, d = private_key
    return pow(ciphertext, d, n)

# Example usage:
pub_key, priv_key = rsa_keygen(16)
msg = 123
ciphertext = rsa_encrypt(msg, pub_key)
decrypted_msg = rsa_decrypt(ciphertext, priv_key)

ciphertext, decrypted_msg

(17855, 123)

## Prosta faktoryzacja brute force

In [3]:
def brute_force_factorization(n):
    factors = []
    for i in range(2, int(n**0.5)+1):
        while n % i == 0:
            factors.append(i)
            n //= i
    if n > 1:
        factors.append(n)
    return factors

# Example usage:
brute_force_factorization(91)  # returns [7, 13]

[7, 13]

# Proste ataki na RSA

## Faktoryzacja Fermata

Faktoryzacja Fermata liczby nieparzystej $n$ opiera się na znalezieniu pary liczb $a,b$ takich, że $n=a^2-b^2$. Wtedy od razu otrzymujemy faktoryzację $$n=(a+b)(a-b).$$

Dla dowolnej złożonej liczby nieparzystej (tzn. liczby nieparzystej niebędącej liczbą pierwszą) znajdziemy takie liczby $a,b$. W szczególności, w przypadku $n=pq$ (jak w RSA) mamy $$a=\frac{p+q}{2},\qquad b=\frac{p-q}{2}.$$

#### Krok 0.
Sprawdzamy, czy $\sqrt{n}$ jest liczbą naturalną. Jeżeli tak - znaleźliśmy faktoryzację i koniec algorytmu.
#### Krok 1.
Definiujemy zmienne pomocnicze
$$a = \left\lceil\sqrt{n}\right\rceil,\qquad
b^2 = a^2 - n.$$
#### Krok 2.
Jeżeli $\sqrt{b^2}$ jest liczbą całkowitą, to kończymy algorytm i zwracamy $a$ oraz $b=\sqrt{b^2}$. Jeżeli nie, to aktualizujemy zmienne:
$$a=a + 1,\qquad b^2=a^2 - n.$$


### Ćwiczenie 2.
Zaimplementuj atak z użyciem faktoryzacji Fermata i przetestuj go na swojej implementacji RSA. Jeżeli atak zakończył się sukcesem - popraw implementację kryptosystemu. Jakie założenia powinny spełniać parametry kryptosystemu, aby atak się nie powiódł?

In [4]:
def rsa_crack(public_key):
    n, e = public_key
    factors = brute_force_factorization(n)
    if len(factors) != 2:
        raise ValueError("RSA modulus factorization failed")
    p, q = factors
    phi_n = (p - 1) * (q - 1)
    d = mod_inverse(e, phi_n)
    return (n, d)

# Example usage:
pub_key, priv_key = rsa_keygen(8)
cracked_key = rsa_crack(pub_key)

priv_key, cracked_key

((143, 113), (143, 113))

Parametry RSA powinny spełniać następujące założenia:
- Liczby pierwsze p oraz q muszą być duże. Typowe zalecenia to przynajmniej 1024-bitowe liczby pierwsze.
- Liczby pierwsze p i q powinny się znacząco różnić. Najlepiej, gdy mają wyraźnie różną długość bitową i dużą różnicę wartości, co minimalizuje skuteczność faktoryzacji Fermata.
- Nie powinny być one zbyt blisko siebie liczbowo. Atak Fermata wykorzystuje fakt, że p \approx q. Gdy odległość jest znaczna, atak staje się czasochłonny.


In [18]:
import time

def rsa_keygen_secure(bit_length=128):
    half_bits = bit_length // 2
    p = generate_prime(half_bits)
    q = generate_prime(half_bits)

    # Enforce large difference between p and q
    while abs(p - q) < 2**(half_bits - 5):
        q = generate_prime(half_bits)

    n = p * q
    phi_n = (p - 1) * (q - 1)

    e = 65537
    if gcd(e, phi_n) != 1:
        e = 3
    d = mod_inverse(e, phi_n)
    return (n, e), (n, d)

def fermat_factorization(n, timeout=5):
    start_time = time.time()
    a = int(n ** 0.5) + 1

    while time.time() - start_time < timeout:
        b2 = a * a - n
        b = int(b2 ** 0.5)
        if b * b == b2:
            p, q = a + b, a - b
            if p * q == n:
                return p, q
        a += 1
    return None  # timeout reached

def rsa_crack_with_timeout(public_key, timeout=5):
    n, e = public_key
    factors = fermat_factorization(n, timeout)
    if factors is None:
        print("Faktoryzacja Fermata nie powiodła się w określonym czasie.")
        return None
    p, q = factors
    phi_n = (p - 1)*(q - 1)
    d = mod_inverse(e, phi_n)
    return (n, d)

# Example (proper) usage:
pub_key, priv_key = rsa_keygen_secure(96)  # 96 bits total modulus size
start = time.time()
cracked_key = rsa_crack_with_timeout(pub_key, timeout=10)
end = time.time()

print(f"Czas trwania ataku: {end - start:.2f} sekund")
print("Odzyskany klucz prywatny:", cracked_key)

Faktoryzacja Fermata nie powiodła się w określonym czasie.
Czas trwania ataku: 10.00 sekund
Odzyskany klucz prywatny: None


# Ataki nie wykorzystujące faktoryzacji

## Atak naiwny

Jeżeli wykładnik $e$ jest niewielki oraz dla wiadomości $m$ wartość $m^e<n$, to $c=m^e\mod n=m^e$. Możemy zatem odzyskać $m$ obliczając $m=\sqrt[e]{c}$.

### Ćwiczenie 3.
Sprawdź, czy Twoja implementacja RSA jest podatna na powyższy atak wykonując co najmniej 1000 prób. Jeżeli którakolwiek zakończyła się powodzeniem - popraw implementację. Jakie założenia powinny spełniać parametry kryptosystemu, żeby atak nie miał szans powodzenia?

In [23]:
def integer_nth_root(x, n):
    """Efficient integer nth-root calculation using binary search."""
    low, high = 0, x
    while low <= high:
        mid = (low + high) // 2
        mid_n = mid ** n
        if mid_n == x:
            return mid
        elif mid_n < x:
            low = mid + 1
        else:
            high = mid - 1
    return high  # closest integer root from below

def naive_rsa_attack(ciphertext, e):
    return integer_nth_root(ciphertext, e)

# Example demonstration of vulnerability:
pub_key, priv_key = rsa_keygen_secure(64)
n, e = pub_key

# Create a message smaller than the eth-root of n:
msg = 42  # small message to ensure m^e < n
ciphertext = rsa_encrypt(msg, pub_key)

# Check condition for vulnerability explicitly:
if msg ** e < n:
    recovered_msg = naive_rsa_attack(ciphertext, e)
    print(f"Odzyskana wiadomość: {recovered_msg}")
else:
    print("Warunek m^e < n nie jest spełniony, atak nie działa.")

Warunek m^e < n nie jest spełniony, atak nie działa.


In [25]:
def check_naive_attack(trials=1000, bit_length=64, e=3):
    successes = 0
    for _ in range(trials):
        pub_key, priv_key = rsa_keygen_secure(bit_length)
        n, _ = pub_key
        msg = random.randint(2, integer_nth_root(n, e) - 1)  # Ensure m^e < n
        ciphertext = rsa_encrypt(msg, pub_key)
        recovered_msg = naive_rsa_attack(ciphertext, e)
        if recovered_msg == msg:
            successes += 1
            print(f"Sukces ataku! Wiadomość odzyskana: {recovered_msg}")
            break
    if successes == 0:
        print("Atak naiwny nie powiódł się w 1000 próbach.")

# Example run:
check_naive_attack()

Atak naiwny nie powiódł się w 1000 próbach.


## Atak Wienera

Atak Wienera odtwarza klucz prywatny $d$ z klucza publicznego $(n,e)$ bez konieczności faktoryzacji $n$.

#### Krok 1.
Przeksztacamy ułamek $\frac{e}{n}$ na ułamek łańcuchowy postaci $${\displaystyle x=a_{0}+{\cfrac {1}{a_{1}+{\cfrac {1}{a_{2}+{\cfrac {1}{a_{3}+{\cfrac {1}{a_{4}+\ddots \,\frac{1}{a_k}}}}}}}}}}$$o standardowej reprezentacji $[a_0;a_1,a_2,...,a_{k−2},a_{k−1},a_k]$.

#### Krok 2.
Dla każdego reduktu ułamka łańcuchowego, tzn. ułamka łańcuchowego postaci $[a_0]$, $[a_0;a_1]$, $[a_0;a_1,a_2]$,...,$[a_0;a_1,a_2,...,a_{k−2},a_{k−1},a_k]$ sprawdzamy, czy jest on rozwinięciem ułamka $\frac{s}{d}$ w ułamek łańcuchowy dla pewnej stałej $s$:
- niech $l$ oznacza licznik a $m$ - mianownik reduktu
- sprawdź, czy mianownik $m$ jest nieparzysty. Jeżeli nie - przejdź do sprawdzania kolejnego reduktu.
- sprawdź, czy $em=1\mod l$. Jeżeli nie - przejdź do sprawdzania kolejnego reduktu.
- zdefiniuj wielomian pomocniczy $$Q(x)=x^2-(n-\frac{em-1}{l}+1)x+n.$$Jeżeli pierwiastki tego wielomianu są liczbami całkowitymi, to aktualny mianownik $m$ reduktu jest szukanym kluczem prywatnym $d$.


Atak Wienera niekoniecznie musi zakończyć się sukcesem - jeżeli sprawdzimy wszystkie redukty i żaden nie jest rozwinięciem $\frac{s}{d}$, to dany zestaw parametrów RSA jest odporny na ten atak.

### Ćwiczenie 4.

Sprawdź, czy Twoja implementacja RSA jest podatna na atak Wienera wykonując co najmniej 100 prób. Jeżeli którakolwiek zakończyła się powodzeniem - popraw implementację. Jakie założenia powinny spełniać parametry kryptosystemu, żeby atak nie miał szans powodzenia?

Założenia kryptosystemu:
- Klucz prywatny d musi być duży względem modułu n. Najlepiej, aby spełniał: d > n^{0.292}
- W praktyce standardowo wybiera się publiczny wykładnik e = 65537 i losowe, duże liczby pierwsze.
- Klucz prywatny nie powinien być mały ani szczególnie “łatwy” do odgadnięcia.

In [21]:
from fractions import Fraction
from math import isqrt

def continued_fraction(e, n):
    cf = []
    while n:
        cf.append(e // n)
        e, n = n, e % n
    return cf

def convergents(cf):
    convs = []
    for i in range(len(cf)):
        frac = Fraction(0)
        for a in reversed(cf[:i+1]):
            frac = Fraction(1, frac + a) if frac else Fraction(a)
        convs.append((frac.numerator, frac.denominator))
    return convs

def wiener_attack(e, n):
    cf = continued_fraction(e, n)
    convs = convergents(cf)
    
    for k, d in convs:
        if k == 0 or d <= 1:
            continue  # avoid trivial solutions
        
        # check for integer phi
        if (e * d - 1) % k != 0:
            continue
        
        phi_n = (e * d - 1) // k
        s = n - phi_n + 1
        discr = s * s - 4 * n
        
        if discr >= 0:
            sqrt_discr = isqrt(discr)
            if sqrt_discr * sqrt_discr == discr:
                return d  # valid d found

    return None

# Reliable checking over 1000 attempts:
def check_wiener_vulnerability(trials=1000, bit_length=64):
    successes = 0
    for _ in range(trials):
        pub_key, priv_key = rsa_keygen_secure(bit_length)
        n, e = pub_key
        d = wiener_attack(e, n)
        if d and d == priv_key[1]:
            successes += 1
            print(f"Atak Wienera zakończył się powodzeniem! d = {d}")
            break  # can stop after first success
    if successes == 0:
        print("Żaden atak Wienera nie zakończył się powodzeniem w 1000 próbach.")

# Example run:
check_wiener_vulnerability()

Żaden atak Wienera nie zakończył się powodzeniem w 1000 próbach.
